### Replenishment method 1: Full Storage Capacity
On Day n-1: 
- input inventory on day n-1
- order quantity to reach max capacity

In [1]:
# Initiate inventory records
def inventory_init(df, capacity_n, dict_cap, list_dates, n_day_start):
    # Create first column with max capacity
    df_inv = pd.DataFrame((df.groupby(['store', 'item'])['sales'].max() // 50 + capacity_n - 1) * 50)
    df_inv.columns = ['capacity']
    df_inv = df_inv.reset_index()
    
    # Inventory Day 1
    df_inv[list_dates[n_day_start - 1]] = df_inv['capacity']
    df_inv = df_inv.set_index(['store', 'item']).T
    
    return df_inv

In [2]:
# Create DataFrame of storage capacity per item
def capacity_df(df, capacity_n):
    # Capacity DataFrame = n times maximum sales
    df_cap = pd.DataFrame(df.groupby(['item'])['sales'].max())
    # Name the column
    df_cap.columns = ['sales_max']
    df_cap['capacity'] = (df_cap['sales_max'] // 50 + capacity_n - 1) * 50
    
    # Dictionary 
    dict_cap = dict(zip(df_cap.index.values, df_cap.capacity.values))
    df_cap = df_cap.reset_index().set_index(['item', 'sales_max'])
    return df_cap, dict_cap

In [3]:
# Capacity per item
capacity_n = 10
rolling_ndays = 8
df_cap, dictcap = capacity_df(df_xgb, capacity_n)

# Inventory Records
df_inv = inventorya_init(df_xgb, capacity_n, dict_cap, list_dates, rolling_ndays)

# Forecasts DataFrames
df_roll, df_ft_n, df_rm_n = roll_ft_df(df_sales, df_fcst, 8, frcst_n_days)

# Replenishment days
days_reappro = ['Monday', 'Wednesday', 'Friday', 'Sunday']
df_xgb['replen'] = df_xgb.jour.map(lambda t: t in days_reappro)

# Dictionary with date: replenishment_bool
dict_rep = dict(zip(df_xgb.date.dt.date.values, df_xgb.replen.values))

# Create DataFrames to track replenishment
# Inventory Records per day at each step: Sales Reduction, Replenishment
df_invday = df_inv.copy()
# Inventory is PositiveA: After sales reduction >= 0
df_invpos = df_inv.copy()*0
# Replenishment record per day: Replenishment_Qty = capacity - inventory level
df_repm1 = df_inv.copy()

df_inv.head()

NameError: name 'df_xgb' is not defined

### Method 1
Demand = max capacity - day n-1 inventory (9 PM)

In [4]:
def rep_m1(df_repm1, n_day, df_invday, dict_rep, list_dates):
    # Calculate Replenishment Qty
    if (dict_rep[list_dates[n_day - 1]] == True):
        df_repm1.loc[list_dates[n_day - 1]] = (df_invday.loc['capacity']) - (df_invday.loc[list_dates[n_day-1]])
    else:
        df_repm1.loc[list_dates[n_day - 1]] = 0
        
    # Apply replenishment qty to inventory
    df_invday.loc[list_dates[n_day - 1]] = df_invday.loc[list_dates[n_day-1]] + df_repm1.loc[list_dates[n_day-1]]
    
    return df_repm1, df_invday
    

### Run simulation by day

In [5]:
def simul_rem1(rolling_ndays, days, list_dates, dict_rep, df_invday, df_sales, df_repm1, df_invpos):
    
    # First for n_day = 1
    n_day = rolling_ndays
    
    # Sales Substraction: Inventory = Inventory - Sales
    df_invday.loc[list_dates[n_day-1]] = df_invday.loc[list_dates[n_day-1]]  - df_sales.loc[list_dates[n_day-1]]
    
    # Replenishment Method 1: Replenish to Capacity 
    df_repm1.loc[list_dates[n_day-1]] = df_invday.loc['capacity']  - df_invday.loc[list_dates[n_day-1]]
    df_invday.loc[list_dates[n_day-1]] = df_invday.loc[list_dates[n_day-1]] + df_repm1.loc[list_dates[n_day-1]]
    
    # Is Inventory Positive after Sales Reduction ? [1: True, 0: False]
    df_invpos.loc[list_dates[n_day-1]] = (df_invday.loc[list_dates[n_day-1]] < 0) * 1
    
    for n_day in range(rolling_ndays + 1, days + rolling_ndays): #len(list_dates) +1
        # Sales Reduction: Inventory = Inventory - Sales
        df_invday = sales_reduction(df_invday, df_sales, n_day, list_dates)

        # Replenishment Method 1: Replenish to Capacity 
        df_repm1, df_invday = rep_m1(df_repm1, n_day, df_invday, dict_rep, list_dates)

        # Is Inventory Positive after Sales Reduction ? [1: True, 0: False]
        df_invpos = inv_pos(df_invpos, df_invday, n_day, list_dates)
    
    # Calculate Inventory level 
    df_invday['total_inv'] = df_invday.sum(axis = 1)
    df_invday['%total_inv'] = (100*df_invday.iloc[1:, -1].div(df_invday.iloc[0, -1])).round(2)
    return df_invday, df_repm1